# Transform the data to work with Snorkel: Part 2 - Event Role

Here we will do most of the work creating a labeling model that assigns labels to argument roles in event mentions.
We need to create a row for each pair of trigger and entity mention.

For this we need to create 2 additional columns:
- trigger_id
- argument_id

Everything else we can pull from the other columns using Snorkel preprocessor functions.

In [ ]:
import sys
sys.path.append("../")
import warnings
import pickle
from pathlib import Path
from wsee.utils import utils
from wsee.data import pipeline

warnings.filterwarnings(action='once')

DATA_DIR = '/Users/phuc/data/daystream_corpus'  # replace path to corpus

### SD4M Relation/ Event Arguments

| Number | Code       | Description                                                                 |
|--------|------------|-----------------------------------------------------------------------------|
| -1     | ABSTAIN    | No vote, for Labeling Functions                                             |
| 0      | location   | Required argument for all events denoting the location.                     |
| 1      | delay      | Optional argument denoting the delay associated with the event.             |
| 2      | direction  | Optional argument denoting the direction associated with the event.         |
| 3      | start_loc  | Optional argument denoting the starting location associated with the event. |
| 4      | end_loc    | Optional argument denoting the ending location associated with the event.   |
| 5      | start_date | Optional argument denoting the start date associated with the event.        |
| 6      | end_date   | Optional argument denoting the end date associated with the event.          |
| 7      | cause      | Optional argument (trigger) denoting the cause associated with the event.   |
| 8      | jam_length | Optional argument denoting the jam length of a traffic jam event.           |
| 9      | route      | Optional argument denoting the route affected by a canceled stop event.     |
| 10     | no_arg     | No argument relation with the specified trigger.                            |

In [ ]:
loaded_data = pipeline.load_data(DATA_DIR)
sd_train = loaded_data['train']
sd_dev = loaded_data['dev']
sd_test = loaded_data['test']

daystream = loaded_data['daystream']

In [ ]:
sd_train.head()

## Step 1: Create one row for each trigger-entity pair (event role)

In [ ]:
dataframe_file = DATA_DIR + '/pickled_sd_train_role_examples'
pickled_dataframe_file = Path(dataframe_file + '.pkl')

df_sd_train = None
Y_sd_train = None

if pickled_dataframe_file.exists():
    with open(pickled_dataframe_file, 'rb') as pickled_dataframe:
        df_sd_train, Y_sd_train = pickle.load(pickled_dataframe)
else:
    df_sd_train, Y_sd_train = pipeline.build_event_role_examples(sd_train)
    with open(pickled_dataframe_file, 'wb') as pickled_dataframe:
        pickle.dump((df_sd_train, Y_sd_train), pickled_dataframe)

In [ ]:
dataframe_file = DATA_DIR + '/pickled_sd_dev_role_examples'
pickled_dataframe_file = Path(dataframe_file + '.pkl')

df_sd_dev = None
Y_sd_dev = None

if pickled_dataframe_file.exists():
    with open(pickled_dataframe_file, 'rb') as pickled_dataframe:
        df_sd_dev, Y_sd_dev = pickle.load(pickled_dataframe)
else:
    df_sd_dev, Y_sd_dev = pipeline.build_event_role_examples(sd_dev)
    with open(pickled_dataframe_file, 'wb') as pickled_dataframe:
        pickle.dump((df_sd_dev, Y_sd_dev), pickled_dataframe)

In [ ]:
from wsee import ROLE_LABELS
print(ROLE_LABELS)

## Step 2: Explore the data

In [ ]:
from wsee.preprocessors.preprocessors import *
from wsee.data import explore

We can apply all our preprocessors on our data and see if we can find something interesting for our labeling functions. Let's first sample the SD4M training data, which is labeled.

In [ ]:
dataframe_file = DATA_DIR + '/pickled_labeled_sd4m_role_examples'
pickled_dataframe_file = Path(dataframe_file + '.pkl')

labeled_sd4m_roles = None

if pickled_dataframe_file.exists():
    with open(pickled_dataframe_file, 'rb') as pickled_dataframe:
        labeled_sd4m_roles = pickle.load(pickled_dataframe)
else:
    labeled_sd4m_roles = explore.add_labels(df_sd_train, Y_sd_train)
    labeled_sd4m_roles = explore.apply_preprocessors(labeled_sd4m_roles, [pre_between_tokens, pre_between_distance])
    labeled_sd4m_roles = explore.add_event_types(labeled_sd4m_roles)
    labeled_sd4m_roles = explore.add_event_arg_roles(labeled_sd4m_roles)
    with open(pickled_dataframe_file, 'wb') as pickled_dataframe:
        pickle.dump(labeled_sd4m_roles, pickled_dataframe)

Let's first take a look at the trigger and argument text, and the entity types!

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
# explore.sample_data(labeled_sd4m_roles[labeled_sd4m_roles['label']==6], columns=['text', 'between_tokens', 'trigger', 'argument', 'between_distance', 'label', 'event_types', 'event_arg_roles'])

Now we can collect the most frequent trigger-argument pairs per class.

In [ ]:
n = 100
filtered_sd4m_roles = labeled_sd4m_roles[labeled_sd4m_roles['label'] != 10]
class_pairs = {}
print(f"Number of event-roles: {len(labeled_sd4m_roles)}\n")
for idx, class_name in enumerate(ROLE_LABELS):
    class_sd4m_roles = labeled_sd4m_roles[labeled_sd4m_roles['label'] == idx]
    print(f"{class_name}: {len(class_sd4m_roles)} instances")

Only checking the argument text probably does not give us much, but it shall serve as an example.

## Step 3: Evaluate the labeling functions on the SD4M training data

In [ ]:
from wsee.labeling import event_argument_role_lfs as role_lfs

### Apply the labeling functions

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [
    role_lfs.lf_location_adjacent_markers,
    role_lfs.lf_location_adjacent_trigger_verb,
    role_lfs.lf_location_beginning_street_stop_route,
    role_lfs.lf_location_first_sentence_street_stop_route,
    role_lfs.lf_location_first_sentence_priorities,
    role_lfs.lf_delay_event_sentence,
    role_lfs.lf_delay_preceding_arg,
    role_lfs.lf_delay_preceding_trigger,
    role_lfs.lf_direction_markers,
    role_lfs.lf_direction_markers_order,
    role_lfs.lf_direction_pattern,
    role_lfs.lf_direction_markers_pattern_order,
    role_lfs.lf_start_location_type,
    role_lfs.lf_start_location_nearest,
    role_lfs.lf_start_location_preceding_arg,
    role_lfs.lf_end_location_type,
    role_lfs.lf_end_location_nearest,
    role_lfs.lf_end_location_preceding_arg,
    role_lfs.lf_start_date_type,
    role_lfs.lf_start_date_replicated,
    role_lfs.lf_start_date_first,
    role_lfs.lf_start_date_adjacent,
    role_lfs.lf_end_date_type,
    role_lfs.lf_end_date_replicated,
    role_lfs.lf_cause_type,
    role_lfs.lf_cause_replicated,
    role_lfs.lf_cause_order,
    role_lfs.lf_distance_type,
    role_lfs.lf_distance_nearest,
    role_lfs.lf_distance_order,
    role_lfs.lf_route_type,
    role_lfs.lf_route_type_order,
    role_lfs.lf_route_type_order_between_check,
    role_lfs.lf_delay_earlier_negative,
    role_lfs.lf_date_negative,
    role_lfs.lf_not_an_event,
    role_lfs.lf_somajo_separate_sentence,
    role_lfs.lf_overlapping,
    role_lfs.lf_too_far_40,
    role_lfs.lf_multiple_same_event_type,
    role_lfs.lf_event_patterns,
    role_lfs.lf_event_patterns_general_location
]

applier = PandasLFApplier(lfs)

In [ ]:
L_sd_train = applier.apply(df_sd_train)

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L_sd_train, lfs).lf_summary(Y_sd_train)

## Step 4: Error Analysis

In [ ]:
from wsee.labeling import error_analysis

In [ ]:
relevant_rows = labeled_sd4m_roles.iloc[L_sd_train[:, 0] == 0]
print(len(relevant_rows))
relevant_rows.sample()[['text', 'trigger', 'argument', 'label', 'event_types', 'event_arg_roles']]

In [ ]:
error_analysis.sample_fp(labeled_df=labeled_sd4m_roles, lf_outputs=L_sd_train, lf_index=0, label_of_interest=0, sample_size=1)[['between_tokens', 'trigger', 'argument', 'somajo_doc', 'label', 'event_types', 'event_arg_roles']]

In [ ]:
error_analysis.sample_abstained_instances(labeled_df=labeled_sd4m_roles, lf_outputs=L_sd_train, lf_index=19, label_of_interest=5, sample_size=1)[['text', 'between_tokens', 'trigger', 'argument', 'label', 'event_types', 'event_arg_roles']]

In [ ]:
error_analysis.sample_abstained_instances(labeled_df=labeled_sd4m_roles, lf_outputs=L_sd_train, lf_index=0, label_of_interest=0, sample_size=1)[['text', 'between_tokens', 'trigger', 'argument', 'label', 'event_types']]

## Step 5: Train the Labeling model and label the data

In [ ]:
dataframe_file = DATA_DIR + '/pickled_daystream_role_examples'
pickled_dataframe_file = Path(dataframe_file + '.pkl')

df_daystream = None
Y_daystream = None

if pickled_dataframe_file.exists():
    with open(pickled_dataframe_file, 'rb') as pickled_dataframe:
        df_daystream, Y_daystream = pickle.load(pickled_dataframe)
else:
    df_daystream, Y_daystream = pipeline.build_event_role_examples(daystream)
    with open(pickled_dataframe_file, 'wb') as pickled_dataframe:
        pickle.dump((df_daystream, Y_daystream), pickled_dataframe)
if 'event_roles' in df_daystream:
    df_daystream.drop('event_roles', axis=1, inplace=True)

In [ ]:
L_daystream = applier.apply(df_daystream)

In [ ]:
from snorkel.labeling import LFAnalysis
LFAnalysis(L_daystream, lfs).lf_summary()

In [ ]:
from snorkel.labeling import LabelModel

daystream_model = LabelModel(cardinality=11, verbose=True)
daystream_model.fit(L_train=L_daystream, n_epochs=5000, log_freq=500, seed=12345, Y_dev=Y_sd_train)

In [ ]:
daystream_model_acc = daystream_model.score(L=L_sd_train, Y=Y_sd_train, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {daystream_model_acc * 100:.1f}%")

In [ ]:
daystream_probs = daystream_model.predict_proba(L=L_daystream)

In [ ]:
labeled_daystream_with_abstains = pipeline.merge_event_role_examples(df_daystream, utils.zero_out_abstains(daystream_probs, L_daystream))
labeled_daystream_with_abstains.reset_index(level=0).to_json(DATA_DIR + "/save_daystreamv6_roles_with_abstains.jsonl", orient='records', lines=True, force_ascii=False)

## Step 7: Daystream Snorkel Labeling Check

To look at the daystream labeling it would be best to remove the abstains.

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_daystream_filtered, probs_daystream_filtered = filter_unlabeled_dataframe(
    X=df_daystream, y=daystream_probs, L=L_daystream
)

In [ ]:
df_daystream_filtered['role_probs'] = list(probs_daystream_filtered)
df_daystream_filtered['most_probable_class'] = [ROLE_LABELS[label_idx] for label_idx in probs_daystream_filtered.argmax(axis=1)]
df_daystream_filtered['max_class_prob'] = ["{:.2f}".format(class_prob) for class_prob in probs_daystream_filtered.max(axis=1)]

In [ ]:
for role_class in ROLE_LABELS:
    print(f"{role_class}: {len(df_daystream_filtered[df_daystream_filtered['most_probable_class'] == role_class])} instances")

In [ ]:
df_daystream_filtered[df_daystream_filtered['most_probable_class'] == 'route'].sample(1)[['text', 'trigger', 'argument', 'most_probable_class', 'max_class_prob', 'role_probs']]